In [1]:
import datetime
import random
import numpy as np
import pandas_datareader.data as web
import tensorflow as tf

定数一覧

|変数名|定数の説明|
|:----|:----|
|training_days|推定に用いるデータの日数|
|learning_rate|学習率|
|num_of_input_nodes|入力層のノード数|
|num_of_hidden_nodes|隠れ層のノード数|
|num_of_output_nodes|出力層のノード数|
|forget_bias|忘却ゲートで使われるバイアス|
|size_of_mini_batch|ミニバッチのサイズ|
|num_of_training_epochs|学習のエポック数|

In [15]:
training_days = 30
learning_rate = 0.01
num_of_input_nodes = 1
num_of_hidden_nodes = 100
num_of_output_nodes = 1
forget_bias = 0.8
size_of_mini_batch = 50
num_of_training_epochs = 50000

In [3]:
random.seed(0)
np.random.seed(0)
tf.set_random_seed(0)

pandas を使って Yahoo! Finance から日経225の日足データを15年分取得し学習用データとする

In [4]:
def close_prices_to_data(returns):
    X = []
    t = []
    for i in range(0, len(returns) - training_days - 1):
        X.append(returns[i:i+training_days])
        t.append(returns[i+training_days])
    X = np.array(X)
    t = np.array(t)
    return X, t

def create_test_data(training_days):
    start = datetime.datetime(2000, 1, 1)
    end = datetime.datetime(2016, 1, 1)
    n225 = web.DataReader("^N225", 'yahoo', start, end)
    n225_close = n225['Adj Close'].values
    n225_returns = list(map(lambda b, a: 250 * (a - b) / b, n225_close, n225_close[1:]))
    return close_prices_to_data(n225_returns)

def create_training_data(training_days):
    start = datetime.datetime(2016, 1, 1)
    end = datetime.datetime(2017, 1, 1)
    n225 = web.DataReader("^N225", 'yahoo', start, end)
    n225_close = n225['Adj Close'].values
    n225_returns = list(map(lambda b, a: 250 * (a - b) / b, n225_close, n225_close[1:]))
    return close_prices_to_data(n225_returns)

以下のコードは記事 [TensorFlowのRNNを基本的なモデルで試す](http://qiita.com/yukiB/items/f6314d2861fc8d9b739f) を参考

In [5]:
def inference(input_ph, istate_ph):
    with tf.name_scope("inference") as scope:
        weight1_var = tf.Variable(tf.truncated_normal(
            [num_of_input_nodes, num_of_hidden_nodes], stddev=0.1), name="weight1")
        weight2_var = tf.Variable(tf.truncated_normal(
            [num_of_hidden_nodes, num_of_output_nodes], stddev=0.1), name="weight2")
        bias1_var = tf.Variable(tf.truncated_normal([num_of_hidden_nodes], stddev=0.1), name="bias1")
        bias2_var = tf.Variable(tf.truncated_normal([num_of_output_nodes], stddev=0.1), name="bias2")

        in1 = tf.transpose(input_ph, [1, 0, 2])
        in2 = tf.reshape(in1, [-1, num_of_input_nodes])
        in3 = tf.matmul(in2, weight1_var) + bias1_var
        in4 = tf.split(0, training_days, in3)

        cell = tf.nn.rnn_cell.BasicLSTMCell(
            num_of_hidden_nodes, forget_bias=forget_bias, state_is_tuple=False)
        rnn_output, states_op = tf.nn.rnn(cell, in4, initial_state=istate_ph)
        output_op = tf.matmul(rnn_output[-1], weight2_var) + bias2_var

        # Add summary ops to collect data
        w1_hist = tf.summary.histogram("weights1", weight1_var)
        w2_hist = tf.summary.histogram("weights2", weight2_var)
        b1_hist = tf.summary.histogram("biases1", bias1_var)
        b2_hist = tf.summary.histogram("biases2", bias2_var)
        output_hist = tf.summary.histogram("output",  output_op)
        results = [weight1_var, weight2_var, bias1_var,  bias2_var]
        return output_op, states_op, results

In [6]:
def loss(output_op, supervisor_ph):
    with tf.name_scope("loss") as scope:
        square_error = tf.reduce_mean(tf.square(output_op - supervisor_ph))
        loss_op = square_error
        tf.summary.scalar("loss", loss_op)
        return loss_op

In [7]:
def training(loss_op):
    with tf.name_scope("training") as scope:
        training_op = optimizer.minimize(loss_op)
        return training_op

In [8]:
def get_batch(batch_size, X, t):
    rnum = [random.randint(0, len(X) - 1) for x in range(batch_size)]
    xs = np.array([[[y] for y in list(X[r])] for r in rnum])
    ts = np.array([[t[r]] for r in rnum])
    return xs, ts

In [13]:
def print_accuracy(output_op, X, t):
    Xs = np.array([[[y] for y in list(X[r])] for r in range(0, len(X))])
    ts = np.array([[t[r]] for r in range(0, len(t))])
    pred_dict = {
        input_ph:  Xs,
        supervisor_ph: ts,
        istate_ph:    np.zeros((len(X), num_of_hidden_nodes * 2)),
    }
    output = sess.run([output_op], feed_dict=pred_dict)

    opt = (output * ts)[0]
    total = sum([1 if x[0] > 0 else 0 for x in opt])
    print("accuracy %f" % (total / float(len(ts))))
    return output

In [16]:
X_test, t_test = create_test_data(training_days)
X_training, t_training = create_training_data(training_days)

optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)

with tf.Graph().as_default():
    input_ph = tf.placeholder(tf.float32, [None, training_days, num_of_input_nodes], name="input")
    supervisor_ph = tf.placeholder(tf.float32, [None, num_of_output_nodes], name="supervisor")
    istate_ph = tf.placeholder(tf.float32, [None, num_of_hidden_nodes * 2], name="istate")
    
    output_op, states_op, datas_op = inference(input_ph, istate_ph)
    loss_op = loss(output_op, supervisor_ph)
    training_op = training(loss_op)
    
    summary_op = tf.summary.merge_all()
    init = tf.global_variables_initializer()
    
    with tf.Session() as sess:
        saver = tf.train.Saver()
        summary_writer = tf.summary.FileWriter("/tmp/tensorflow_log", graph=sess.graph)
        sess.run(init)
        
        for epoch in range(num_of_training_epochs):
            inputs, supervisors = get_batch(size_of_mini_batch, X_test, t_test)
            train_dict = {
                input_ph:      inputs,
                supervisor_ph: supervisors,
                istate_ph:     np.zeros((size_of_mini_batch, num_of_hidden_nodes * 2)),
            }
            sess.run(training_op, feed_dict=train_dict)
            
            if (epoch) % 1000 == 0:
                summary_str, train_loss = sess.run([summary_op, loss_op], feed_dict=train_dict)
                print("train#%d, train loss: %e" % (epoch, train_loss))
                summary_writer.add_summary(summary_str, epoch)
                if (epoch) % 5000 == 0:
                    print_accuracy(output_op, X_training, t_training)

        print_accuracy(output_op, X_training, t_training)
        datas = sess.run(datas_op)
        saver.save(sess, "model.ckpt")

train#0, train loss: 1.094412e+01
accuracy 0.549296
train#1000, train loss: 1.953825e+01
train#2000, train loss: 1.172229e+01
train#3000, train loss: 9.834884e+00
train#4000, train loss: 8.167578e+00
train#5000, train loss: 1.065910e+01
accuracy 0.507042
train#6000, train loss: 9.477546e+00
train#7000, train loss: 2.047491e+01
train#8000, train loss: 7.286870e+00
train#9000, train loss: 9.135774e+00
train#10000, train loss: 5.507623e+00
accuracy 0.563380
train#11000, train loss: 7.381664e+00
train#12000, train loss: 6.440878e+00
train#13000, train loss: 3.235647e+00
train#14000, train loss: 4.996283e+00
train#15000, train loss: 1.154611e+00
accuracy 0.507042
train#16000, train loss: 1.660272e+00
train#17000, train loss: 3.218783e-01
train#18000, train loss: 4.666186e-01
train#19000, train loss: 2.035732e-01
train#20000, train loss: 1.007189e-01
accuracy 0.582160
train#21000, train loss: 8.747717e-02
train#22000, train loss: 7.267650e-01
train#23000, train loss: 5.444868e-02
train#24000

トレーニング時の損失関数
![](loss.png)